In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datasets

from surprise import SVD
from surprise import SVDpp
from surprise.accuracy import rmse, mae, fcp
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import GridSearchCV

In [ ]:
#pip install scikit-surprise

In [ ]:
#pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("amazon_us_reviews", 'Books_v1_00')

In [ ]:
df = pd.DataFrame(dataset['train']['marketplace'])
df['customer_id'] = dataset['train']['customer_id']
df['review_id'] = dataset['train']['review_id']
df['product_id'] = dataset['train']['product_id']
df['product_parent'] = dataset['train']['product_parent']
df['product_title'] = dataset['train']['product_title']
df['star_rating'] = dataset['train']['star_rating']
df['product_category'] = dataset['train']['product_category']
df['helpful_votes'] = dataset['train']['helpful_votes']
df['total_votes'] = dataset['train']['total_votes']
df['vine'] = dataset['train']['vine']
df['verified_purchase'] = dataset['train']['verified_purchase']
df['review_headline'] = dataset['train']['review_headline']
df['review_body'] = dataset['train']['review_body']
df['review_date'] = dataset['train']['review_date']


In [ ]:
#df

In [ ]:
newDf = df[['customer_id', 'product_id', 'star_rating']].copy()

In [ ]:
newDf

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(newDf[['customer_id', 'product_id', 'star_rating']], reader)

In [ ]:
del dataset
del df

In [ ]:
param_grid = {'n_factors': [5,10],
              'n_epochs': [5,10],
              'lr_all': [0.005,0.5],
              'reg_all': [0.02]
             }

In [ ]:
grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE'], cv=10, n_jobs=-1)


In [ ]:
grid_search.fit(data)

In [ ]:
print(grid_search.best_score['rmse'])
print(grid_search.best_params)

In [1]:
#Params chosen as best:
#'n_factors': 5,
#'n_epochs': 10,
#'lr_all': 0.005,
#'reg_all': 0.02 


In [ ]:
%%time
trainset=data.build_full_trainset()

In [ ]:
algo=grid_search.best_estimator['rmse']
#using the best parameters given from grid search

In [ ]:
%%time
algo.fit(trainset)
#training the dataset on the model

In [ ]:
%%time
testset=trainset.build_testset()
#builds testset from trainset

In [ ]:
%%time
predictions=algo.test(testset)
#predicting from model

In [ ]:
%%time
rmse(predictions)

#RMSE Score: 0.8817

In [ ]:
mae(predictions)

#MAE Score: 0.6262

In [ ]:
fcp(predictions)

#FCP Score: 0.6184